A notebook to analyze results of energy_py experiments

In [1]:
import os

import numpy as np
import pandas as pd

from energy_py import make_panel_fig

In [2]:
class Results(object):
    def __init__(self, expt_name, agent_name):
        self.expt_name = expt_name
        self.agent_name = agent_name
        
        self.data = self.load_data()
        self.args = self.load_args()
        
        self.stats = self.make_stats()
        
    def load_data(self):
        base = os.path.join('/Users/adam/git/energy_py/energy_py/experiments', 
                            self.expt_name, self.agent_name, 'csvs')
        
        csvs = ['df_ep.csv', 'df_stp.csv' , 'env_info.csv']
        
        data = {name: pd.read_csv(base + '/' + name, index_col=0, parse_dates=True) 
                for name in csvs}
        
        return data
    
    def load_args(self):
        args_path = os.path.join('/Users/adam/git/energy_py/energy_py/experiments', 
                            self.expt_name, self.agent_name, 'args.txt')
        
        args = {}
        with open(args_path) as f:
            for line in f:
                k,v = line.split(',')
                v = v.rstrip()
                args[k] = v
        return args
    
    def make_stats(self):
        eps = self.data['df_ep.csv']
        steps = self.data['df_stp.csv']
        
        self.avg_rew_stp = steps.loc[:, 'reward'].mean()
        
        #  assuming a 5 min step time
        self.avg_annual_rew = self.avg_rew_stp * (12 * 24 * 365)
        
        self.ep_total_rew = eps.loc[:, 'reward'].values
        
        self.action_count = int(steps.loc[:, 'act_0'].sum())
        
        stats = {'average reward per step': self.avg_rew_stp,
                 'annual avg reward': self.avg_annual_rew,
                 'episode rewards': self.ep_total_rew,
                 'flex actions': self.action_count}
        
        return stats
    
    def __repr__(self):
        return 'results_obj_{}'.format(self.agent_name)
    

In [3]:
import pprint

pp = pprint.PrettyPrinter()

class Experiment(object):
    def __init__(self, expt_name, agents):
        self.results = [Results(expt_name, agent) for agent in agents]
        self.check_results()
        
    def check_results(self):
        #  check that the episode lengths were the same by checking the 
        #  length of the last episode
        all(x==self.results[0].data['env_info.csv'].shape[0]
            for x in self.results)
        
    def output_results(self, print_args=False):
        
        for result in self.results:
            print(repr(result))
            if print_args:  pp.pprint(result.args)
            pp.pprint(result.stats)

In [4]:
exp = Experiment('naive_dispatch', ['no_op', 'dispatch', 'random_0', 'random_1'])

In [5]:
exp.output_results()

results_obj_no_op
{'annual avg reward': 0.0,
 'average reward per step': 0.0,
 'episode rewards': array([ 0.]),
 'flex actions': 0}
results_obj_dispatch
{'annual avg reward': 25943.727095280035,
 'average reward per step': 0.24680105684246609,
 'episode rewards': array([ 218565.53125]),
 'flex actions': 13513}
results_obj_random_0
{'annual avg reward': 597.00161613597925,
 'average reward per step': 0.0056792391184929533,
 'episode rewards': array([ 5029.5]),
 'flex actions': 44560}
results_obj_random_1
{'annual avg reward': -967.09099362783707,
 'average reward per step': -0.0091998762711932754,
 'episode rewards': array([-8147.35546875]),
 'flex actions': 44048}


In [6]:
daily_rew = pd.DataFrame()

for result in exp.results:
    env = result.data['env_info.csv']
    daily_rew[repr(result)] = env['reward'].groupby([env.index.month, env.index.day]).sum()
    
    panels = [[res] for res in daily_rew.columns]
    
    f = make_panel_fig(env, panels)


KeyError: 'the label [results_obj_no_op] is not in the [columns]'

In [ ]:
daily_rew.columns

In [ ]:
for a in daily_rew.columns:
    print([a])

In [ ]:
[[a] for a in daily_rew.columns]